In [23]:
import pandas as pd
import numpy as np
import re
import os
import subprocess
import time
from datetime import date, timedelta, datetime
import sqlalchemy as a
import boto3
import s3fs
import pandasql
import sys
from datetime import date, timedelta, datetime
import copy
from io import StringIO
import io

In [24]:
client = boto3.client('athena', region_name = 'us-east-1')

In [25]:
#Function for starting athena query

def run_query(query, database, s3_output):
    print(s3_output)
    client = boto3.client('athena', region_name = 'us-east-1')
    response = client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={
        'Database': database
           },
    ResultConfiguration={
            'OutputLocation': s3_output,
            },
    WorkGroup='insights_team'    
    )
       
    print('Execution ID: ' + response['QueryExecutionId'])
   
    return response

# Before Guardrails data

In [26]:
# # Visits Before Guardrails-  July Month Data- ORC

# os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
# july_vis = wr.athena.read_sql_query(sql="""
# SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
# FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc
# where  lag_bucket_id not in ('-1','7','8')
# and exposure_type = 'IMPRESSION'
# and y = '2022'
# and m >='07'
# and visit_date>='2022-07-01'
# and visit_date<='2022-07-31'
# group by 1,2,3,4
# """, database="saurabhyadav",workgroup='insights_team')

# july_vis.to_csv('s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails/y=2022/m=07/d=01/july_vis.csv',index=False,header=False)


# # Visits Before Guardrails-  August Month Data- ORC

# os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
# aug_vis = wr.athena.read_sql_query(sql="""
# SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
# FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc
# where  lag_bucket_id not in ('-1','7','8')
# and exposure_type = 'IMPRESSION'
# and y = '2022'
# and m >='07'
# and visit_date>='2022-08-01'
# and visit_date<='2022-08-31'
# group by 1,2,3,4
# """, database="saurabhyadav",workgroup='insights_team')

# aug_vis.to_csv('s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails/y=2022/m=08/d=01/aug_vis.csv',index=False,header=False)


In [27]:
date.today()

datetime.date(2022, 9, 21)

In [28]:
for i in range(8,0,-1):
    dt=date.today()-timedelta(i)
    print(dt)

2022-09-13
2022-09-14
2022-09-15
2022-09-16
2022-09-17
2022-09-18
2022-09-19
2022-09-20


In [29]:
# Sept 1 onwards data from raw table

for i in range(8,0,-1):
    dt=dt.today()-timedelta(i)
    print(dt)
    y, m, d = str(dt).split('-')
    y=str(dt.year)
    m=str(dt.month)
    m1=str(dt.month)
    m2=str(dt.month+1)
    d=str(dt.day)
    if (len(m)==1):
        m='0'+m
    if(len(d)==1):
        d='0'+d
    if (len(m1)==1):
        m1='0'+m1
    if (len(m)==1):
        m2='0'+m2
    
    dt2=dt.today()
    y2, mn2, d2 = str(dt2).split('-')
    y2=str(dt2.year)
    mn2=str(dt2.month)
    d2=str(dt2.day)
    if (len(mn2)==1):
        mn2='0'+mn2
    if(len(d2)==1):
        d2='0'+d2
    e1=mn2+d2
    
    database = 'saurabhyadav'
    query="""
    with base2 as 
    (select *,from_unixtime ((header.timestamp-14400000)/1000 ) as vis_dt,
    from_unixtime ((header.timestamp- cmp.header.timestamp)/1000) as lag,cmp.campaign_id as campaign_id,
    cmp.header.request_id as adx_req_id
    from raw_conversion_tracking.conversion_enigma_event_v2_conversiontracking
    CROSS JOIN unnest(ad_exposure) AS adx (cmp)
    CROSS JOIN unnest(poi_attribution) AS poa (po)
    where cmp.is_latest=TRUE 
    and poi_attribution is not null
    and cmp.event_type='IMPRESSION'
    and (cmp.conversion_type='STORE' or cmp.conversion_type is null)
    and y='2022' and m IN ('{}','{}') and concat(m,d) NOT IN ('{}'))
    
    select date(vis_dt) as visit_date,campaign_id,adx_req_id,uid,count(*) as vis_dummy
    from base2 
    where date(vis_dt)=date('{}-{}-{}') and day(lag)<=14
    group by 1,2,3,4
    
    """.format(m1,m2,e1,y,m,d)
    print(query)
    vis = wr.athena.read_sql_query(sql=query, database="saurabhyadav",workgroup='insights_team')
    vis.to_csv('s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails/y={}/m={}/d={}/visits.csv'.format(y,m,d),index=False,header=False)

    #loading the data into the partitions 
    queryforcreatingtable = """MSCK REPAIR TABLE saurabhyadav.Attributed_vis_before_guardrails;"""
    res_us1 = run_query(queryforcreatingtable, database, 's3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails/')
    time.sleep(10)    
    os.system('aws s3 rm s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails --recursive --exclude "*" --include "*.metadata"')
    os.system('aws s3 rm s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails --recursive --exclude "*" --include "*.txt"')
    time.sleep(10)


2022-09-13

    with base2 as 
    (select *,from_unixtime ((header.timestamp-14400000)/1000 ) as vis_dt,
    from_unixtime ((header.timestamp- cmp.header.timestamp)/1000) as lag,cmp.campaign_id as campaign_id,
    cmp.header.request_id as adx_req_id
    from raw_conversion_tracking.conversion_enigma_event_v2_conversiontracking
    CROSS JOIN unnest(ad_exposure) AS adx (cmp)
    CROSS JOIN unnest(poi_attribution) AS poa (po)
    where cmp.is_latest=TRUE 
    and poi_attribution is not null
    and cmp.event_type='IMPRESSION'
    and (cmp.conversion_type='STORE' or cmp.conversion_type is null)
    and y='2022' and m IN ('09','10') and concat(m,d) NOT IN ('0921'))
    
    select date(vis_dt) as visit_date,campaign_id,adx_req_id,uid,count(*) as vis_dummy
    from base2 
    where date(vis_dt)=date('2022-09-13') and day(lag)<=14
    group by 1,2,3,4
    
    


<IPython.core.display.Javascript object>

s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails/
Execution ID: 488ab449-da4d-4932-877d-8f59912b452b
2022-09-14

    with base2 as 
    (select *,from_unixtime ((header.timestamp-14400000)/1000 ) as vis_dt,
    from_unixtime ((header.timestamp- cmp.header.timestamp)/1000) as lag,cmp.campaign_id as campaign_id,
    cmp.header.request_id as adx_req_id
    from raw_conversion_tracking.conversion_enigma_event_v2_conversiontracking
    CROSS JOIN unnest(ad_exposure) AS adx (cmp)
    CROSS JOIN unnest(poi_attribution) AS poa (po)
    where cmp.is_latest=TRUE 
    and poi_attribution is not null
    and cmp.event_type='IMPRESSION'
    and (cmp.conversion_type='STORE' or cmp.conversion_type is null)
    and y='2022' and m IN ('09','10') and concat(m,d) NOT IN ('0921'))
    
    select date(vis_dt) as visit_date,campaign_id,adx_req_id,uid,count(*) as vis_dummy
    from base2 
    where date(vis_dt)=date('2022-09-14') and day(lag)<=14
    group by 1,2,3,4
    
    


<IPython.core.display.Javascript object>

s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails/
Execution ID: 07dd5af0-3fd5-4512-83b0-96dd407cffde
2022-09-15

    with base2 as 
    (select *,from_unixtime ((header.timestamp-14400000)/1000 ) as vis_dt,
    from_unixtime ((header.timestamp- cmp.header.timestamp)/1000) as lag,cmp.campaign_id as campaign_id,
    cmp.header.request_id as adx_req_id
    from raw_conversion_tracking.conversion_enigma_event_v2_conversiontracking
    CROSS JOIN unnest(ad_exposure) AS adx (cmp)
    CROSS JOIN unnest(poi_attribution) AS poa (po)
    where cmp.is_latest=TRUE 
    and poi_attribution is not null
    and cmp.event_type='IMPRESSION'
    and (cmp.conversion_type='STORE' or cmp.conversion_type is null)
    and y='2022' and m IN ('09','10') and concat(m,d) NOT IN ('0921'))
    
    select date(vis_dt) as visit_date,campaign_id,adx_req_id,uid,count(*) as vis_dummy
    from base2 
    where date(vis_dt)=date('2022-09-15') and day(lag)<=14
    group by 1,2,3,4
    
    


...

s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails/
Execution ID: 7d0c8047-f294-4113-ba00-1bfa7b0c9423
2022-09-19

    with base2 as 
    (select *,from_unixtime ((header.timestamp-14400000)/1000 ) as vis_dt,
    from_unixtime ((header.timestamp- cmp.header.timestamp)/1000) as lag,cmp.campaign_id as campaign_id,
    cmp.header.request_id as adx_req_id
    from raw_conversion_tracking.conversion_enigma_event_v2_conversiontracking
    CROSS JOIN unnest(ad_exposure) AS adx (cmp)
    CROSS JOIN unnest(poi_attribution) AS poa (po)
    where cmp.is_latest=TRUE 
    and poi_attribution is not null
    and cmp.event_type='IMPRESSION'
    and (cmp.conversion_type='STORE' or cmp.conversion_type is null)
    and y='2022' and m IN ('09','10') and concat(m,d) NOT IN ('0921'))
    
    select date(vis_dt) as visit_date,campaign_id,adx_req_id,uid,count(*) as vis_dummy
    from base2 
    where date(vis_dt)=date('2022-09-19') and day(lag)<=14
    group by 1,2,3,4
    
    


<IPython.core.display.Javascript object>

s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails/
Execution ID: 5f3863ea-252b-47af-84fa-541cbe54645b
2022-09-20

    with base2 as 
    (select *,from_unixtime ((header.timestamp-14400000)/1000 ) as vis_dt,
    from_unixtime ((header.timestamp- cmp.header.timestamp)/1000) as lag,cmp.campaign_id as campaign_id,
    cmp.header.request_id as adx_req_id
    from raw_conversion_tracking.conversion_enigma_event_v2_conversiontracking
    CROSS JOIN unnest(ad_exposure) AS adx (cmp)
    CROSS JOIN unnest(poi_attribution) AS poa (po)
    where cmp.is_latest=TRUE 
    and poi_attribution is not null
    and cmp.event_type='IMPRESSION'
    and (cmp.conversion_type='STORE' or cmp.conversion_type is null)
    and y='2022' and m IN ('09','10') and concat(m,d) NOT IN ('0921'))
    
    select date(vis_dt) as visit_date,campaign_id,adx_req_id,uid,count(*) as vis_dummy
    from base2 
    where date(vis_dt)=date('2022-09-20') and day(lag)<=14
    group by 1,2,3,4
    
    


<IPython.core.display.Javascript object>

s3://monet-etl/saurabhyadav/PMO-229/Visits_before_guardrails/
Execution ID: 7f27404a-0e9b-48c1-adc7-46e74285fa4c


# After Guardrails data

In [30]:
# # Visits After Guardrails-  July Month Data- store_visitation_intermediate_orc.store_visitation_intermediate_orc_with_guardrails_pmo227

# os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
# july_vis2 = wr.athena.read_sql_query(sql="""
# SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
# FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc_with_guardrails_pmo227
# where  lag_bucket_id not in ('-1','7','8')
# and exposure_type = 'IMPRESSION'
# and y = '2022'
# and m >='07'
# and visit_date>='2022-07-01'
# and visit_date<='2022-07-31'
# group by 1,2,3,4
# """, database="saurabhyadav",workgroup='insights_team')

# july_vis2.to_csv('s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/y=2022/m=07/d=01/july_vis.csv',index=False,header=False)

# # Visits After Guardrails-  Aug Month Data- store_visitation_intermediate_orc.store_visitation_intermediate_orc_with_guardrails_pmo227

# os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
# aug_vis2 = wr.athena.read_sql_query(sql="""
# SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
# FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc_with_guardrails_pmo227
# where  lag_bucket_id not in ('-1','7','8')
# and exposure_type = 'IMPRESSION'
# and y = '2022'
# and m >='07'
# and visit_date>='2022-08-01'
# and visit_date<='2022-08-31'
# group by 1,2,3,4
# """, database="saurabhyadav",workgroup='insights_team')

# aug_vis2.to_csv('s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/y=2022/m=08/d=01/aug_vis.csv',index=False,header=False)


# # Sept 1-5 data from with_guardrails table

# for i in range(12,7,-1):
#     dt=today-timedelta(i)
#     y, m, d = str(dt).split('-')
#     y=str(dt.year)
#     m=str(dt.month)
#     m1=str(dt.month)
#     m2=str(dt.month+1)
#     d=str(dt.day)
#     if (len(m)==1):
#         m='0'+m
#     if(len(d)==1):
#         d='0'+d
#     if (len(m1)==1):
#         m1='0'+m1
#     if (len(m)==1):
#         m2='0'+m2
#     database = 'saurabhyadav'
#     query="""
#     SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
#     FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc_with_guardrails_pmo227
#     where  lag_bucket_id not in ('-1','7','8')
#     and exposure_type = 'IMPRESSION'
#     and y = '2022'
#     and m IN ('{}','{}')
#     and visit_date='{}-{}-{}'
#     group by 1,2,3,4
#     """.format(m1,m2,y,m,d)
#     print(query)
#     vis = wr.athena.read_sql_query(sql=query, database="saurabhyadav",workgroup='insights_team')
#     vis.to_csv('s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/y={}/m={}/d={}/visits.csv'.format(y,m,d),index=False,header=False)

#     #loading the data into the partitions 
#     queryforcreatingtable = """MSCK REPAIR TABLE saurabhyadav.Attributed_vis_after_guardrails;"""
#     res_us1 = run_query(queryforcreatingtable, database, 's3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/')
#     time.sleep(10)    
#     os.system('aws s3 rm s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails --recursive --exclude "*" --include "*.metadata"')
#     os.system('aws s3 rm s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails --recursive --exclude "*" --include "*.txt"')
#     time.sleep(10)



In [31]:
# Sept 6 onwards data from ORC table

for i in range(8,0,-1):
    dt=dt.today()-timedelta(i)
    y, m, d = str(dt).split('-')
    y=str(dt.year)
    m=str(dt.month)
    m1=str(dt.month)
    m2=str(dt.month+1)
    d=str(dt.day)
    if (len(m)==1):
        m='0'+m
    if(len(d)==1):
        d='0'+d
    if (len(m1)==1):
        m1='0'+m1
    if (len(m)==1):
        m2='0'+m2
    database = 'saurabhyadav'
    query="""
    SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
    FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc
    where  lag_bucket_id not in ('-1','7','8')
    and exposure_type = 'IMPRESSION'
    and y = '2022'
    and m IN ('{}','{}')
    and visit_date='{}-{}-{}'
    group by 1,2,3,4
    """.format(m1,m2,y,m,d)
    print(query)
    vis = wr.athena.read_sql_query(sql=query, database="saurabhyadav",workgroup='insights_team')
    vis.to_csv('s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/y={}/m={}/d={}/visits.csv'.format(y,m,d),index=False,header=False)

    #loading the data into the partitions 
    queryforcreatingtable = """MSCK REPAIR TABLE saurabhyadav.Attributed_vis_after_guardrails;"""
    res_us1 = run_query(queryforcreatingtable, database, 's3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/')
    time.sleep(10)    
    os.system('aws s3 rm s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails --recursive --exclude "*" --include "*.metadata"')
    os.system('aws s3 rm s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails --recursive --exclude "*" --include "*.txt"')
    time.sleep(10)



    SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
    FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc
    where  lag_bucket_id not in ('-1','7','8')
    and exposure_type = 'IMPRESSION'
    and y = '2022'
    and m IN ('09','10')
    and visit_date='2022-09-13'
    group by 1,2,3,4
    


<IPython.core.display.Javascript object>

s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/
Execution ID: 48bed0bb-ef64-4ee3-89c2-598936eeffdb

    SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
    FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc
    where  lag_bucket_id not in ('-1','7','8')
    and exposure_type = 'IMPRESSION'
    and y = '2022'
    and m IN ('09','10')
    and visit_date='2022-09-14'
    group by 1,2,3,4
    


<IPython.core.display.Javascript object>

s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/
Execution ID: b613b6e3-c68c-4816-8854-c44930de0f25

    SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
    FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc
    where  lag_bucket_id not in ('-1','7','8')
    and exposure_type = 'IMPRESSION'
    and y = '2022'
    and m IN ('09','10')
    and visit_date='2022-09-15'
    group by 1,2,3,4
    


...

s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/
Execution ID: 25a1a13c-df59-442e-b586-10616c0dc8a6

    SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
    FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc
    where  lag_bucket_id not in ('-1','7','8')
    and exposure_type = 'IMPRESSION'
    and y = '2022'
    and m IN ('09','10')
    and visit_date='2022-09-19'
    group by 1,2,3,4
    


<IPython.core.display.Javascript object>

s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/
Execution ID: bdbd3b5f-c412-44b7-94ee-af17adc4232c

    SELECT date(visit_date) as visit_date,campaign_id,ad_exposure_request_id,visit_uid, sum(visit) as visit
    FROM store_visitation_intermediate_orc.store_visitation_intermediate_orc
    where  lag_bucket_id not in ('-1','7','8')
    and exposure_type = 'IMPRESSION'
    and y = '2022'
    and m IN ('09','10')
    and visit_date='2022-09-20'
    group by 1,2,3,4
    


<IPython.core.display.Javascript object>

s3://monet-etl/saurabhyadav/PMO-229/Visits_after_guardrails/
Execution ID: ceeb95e7-e158-42cc-8341-fa6d4f8ebb20


In [ ]:
for i in range(70,0,-7):
    dt=dt.today()-timedelta(i)
    print(dt)
    p3=(dt - timedelta(days=6))
    print('P3= {}'.format(p3))
    p7=dt
    print('P7= {}'.format(p7))
    p8=(dt-timedelta(days=4))
    print('P8= {}'.format(p8))
    
    year1=str(dt.year)
    month1=str(dt.month)
    day1=str(dt.day)

    if (len(month1)==1):
        month1='0'+month1
    if(len(day1)==1):
        day1='0'+day1


    write_log("", logfile)
    write_log("Dwell time Script Run on {} ".format(date_object.date()), logfile)


    from datetime import datetime
    nowbegining = datetime.now()


    #Function for starting athena query
    def run_query(query, database, s3_output):
        print(s3_output)
        client = boto3.client('athena', region_name = 'us-east-1')
        response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
               },
        ResultConfiguration={
                'OutputLocation': s3_output,
                },
        WorkGroup='insights_team'
        )

        print('Execution ID: ' + response['QueryExecutionId'])

        return response


    from datetime import datetime
    nowbegining7 = datetime.now()
    s3_output_ha = 's3://monet-etl/Suraaj/dwell_time/year={}/month={}/day={}/'.format(year1,month1,day1)
    print(s3_output_ha)

In [ ]:

# script - Importing required libraries
import boto3
import os
import sys
import time
import datetime


client = boto3.client('athena', region_name = 'us-east-1')
logfile = "/Users/saurabhyadav/Desktop/US_Visitation/Weekly_Report/new_script_logs.log"
#f = open(logfile, 'a')           # 'a' will append to an existing file if it exists
def write_log(text, file):
    f = open(file, 'a') 
    f.write("{}\n".format(text))  # write the text to the logfile and move to next line
    return 

database = 'suraaj'

#getting date from user
from datetime import timedelta

#date_entry = input('Enter a date in YYYY-MM-DD format') #-- input using notebook

for i in range(70,0,-7):
    dt=dt.today()-timedelta(i)
    print(dt)
    p3=(dt - timedelta(days=6))
    print('P3= {}'.format(p3))
    p7=dt
    print('P7= {}'.format(p7))
    p8=(dt-timedelta(days=4))
    print('P8= {}'.format(p8))
    
    year1=str(dt.year)
    month1=str(dt.month)
    day1=str(dt.day)

    if (len(month1)==1):
        month1='0'+month1
    if(len(day1)==1):
        day1='0'+day1


    write_log("", logfile)
    write_log("Dwell time Script Run on {} ".format(date_object.date()), logfile)


    from datetime import datetime
    nowbegining = datetime.now()


    #Function for starting athena query
    def run_query(query, database, s3_output):
        print(s3_output)
        client = boto3.client('athena', region_name = 'us-east-1')
        response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
               },
        ResultConfiguration={
                'OutputLocation': s3_output,
                },
        WorkGroup='insights_team'
        )

        print('Execution ID: ' + response['QueryExecutionId'])

        return response


    from datetime import datetime
    nowbegining7 = datetime.now()
    s3_output_ha = 's3://monet-etl/Suraaj/dwell_time/year={}/month={}/day={}/'.format(year1,month1,day1)
    database='suraaj;'
    q7=""" select dwell_time,count(*)
    from
    (select b.dwell_time,a.dt,a.uid,a.poi_hash from
    (
    (SELECT (extract(hour from timestamp)) as local_hour,dt,uid,poi_hash
    from "visit_store"."visitation_hourly_summary_us" 
    where dt between '{p3}' and '{p8}'
    and proximity_mode in (1,2)
        and fence_type = 1 
        and abnormal_req is null 
        group by 1,2,3,4)a
    left join
    (select dt,dwell_hour,uid,poi_hash,dwell_time
    from "dwell_time"."dwell_time_cntry_us"
    where dt between '{p3}' and '{p8}'
    group by 1,2,3,4,5) b
    on a.uid=b.uid
    and a.poi_hash=b.poi_hash
    and a.dt=b.dt
    and a.local_hour =cast(b.dwell_hour as integer)
    )
    group by 1,2,3,4
    )
    group by 1
    having dwell_time=0""".format_map({'p3':p3, 'p8':p8})

    print()
    print("Executing dwell time query: {}".format(q7))
    result7 = run_query(q7, database, s3_output_ha)

    q_visits_status7='QUEUED'
    while(q_visits_status7 in ['QUEUED','RUNNING']):
            time.sleep(1)
            q_visits_status7 = client.get_query_execution(QueryExecutionId = result7['QueryExecutionId'])['QueryExecution']['Status']['State']
    ##### 2nd half of the query
    s3_output_ha = 's3://monet-etl/Suraaj/dwell_time/year={}/month={}/day={}/'.format(year1,month1,day1)
    database='suraaj;'
    q8=""" select dwell_time,count(*)
    from
    (select b.dwell_time,a.dt,a.uid,a.poi_hash from
    (
    (SELECT (extract(hour from timestamp)) as local_hour,dt,uid,poi_hash
    from "visit_store"."visitation_hourly_summary_us" 
    where dt > '{p8}' and dt <='{p7}'
    and proximity_mode in (1,2)
        and fence_type = 1 
        and abnormal_req is null 
        group by 1,2,3,4)a
    left join
    (select dt,dwell_hour,uid,poi_hash,dwell_time
    from "dwell_time"."dwell_time_cntry_us"
    where dt > '{p8}' and dt <='{p7}'
    group by 1,2,3,4,5) b
    on a.uid=b.uid
    and a.poi_hash=b.poi_hash
    and a.dt=b.dt
    and a.local_hour =cast(b.dwell_hour as integer)
    )
    group by 1,2,3,4
    )
    group by 1
    having dwell_time=0""".format_map({'p8':p8, 'p7':p7})

    print()
    print("Executing dwell time query: {}".format(q8))
    result8 = run_query(q8, database, s3_output_ha)

    q_visits_status8='QUEUED'
    while(q_visits_status8 in ['QUEUED','RUNNING']):
            time.sleep(1)
            q_visits_status8 = client.get_query_execution(QueryExecutionId = result8['QueryExecutionId'])['QueryExecution']['Status']['State']


    #######
    #deleting the csv.metadata created in the process

    os.system('aws s3 rm s3://monet-etl/Suraaj/dwell_time/ --recursive --exclude "*" --include "*.metadata"')

    #loading the data into the partitions 
    queryforrepairingtable7 = """MSCK REPAIR TABLE suraaj.dwell_time;"""
    res_us7 = run_query(queryforrepairingtable7, database, 's3://monet-etl/Suraaj/dwell_time/')

    os.system('aws s3 rm s3://monet-etl/Suraaj/dwell_time/ --recursive --exclude "*" --include "*.metadata"')
    os.system('aws s3 rm s3://monet-etl/Suraaj/dwell_time/ --recursive --exclude "*" --include "*.txt"')

    #logs 
    from datetime import datetime
    nowlater7 = datetime.now() 

    print("started at",nowbegining7)

    if  q_visits_status7=='SUCCEEDED':
        print("Dwell time script Succeded for first 3 and ended at ",nowlater7)
        write_log("Dwell time script first 3 Succeded",logfile)
    else:
        print("Dwell time script Failed")
        write_log("Dwell time script Failed {} ".format(date_object.date()), logfile)

    if  q_visits_status8=='SUCCEEDED':
        print("Dwell time script Succeded last 4 and ended at ",nowlater7)
        write_log("Dwell time script Succeded",logfile)
    else:
        print("Dwell time script Failed")
        write_log("Dwell time script last 4Failed {} ".format(date_object.date()), logfile)

